## Tutorial 4 - Isoparametric mappings and numerical integration


In [34]:
import sys
import os
# Add parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
        
from mha021 import *

In [35]:
coords, weights = gauss_integration_rule(ngp=2)

displayvar("coords", coords)
displayvar("weights", weights)


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
def f(ξ):
    return np.cos(ξ) # exact: 2*np.sin(1)
I_exact = 2*np.sin(1)

displayvar("I_{exact}", I_exact)
# I = f(ξ₁)*W₁ + f(ξ₂)*W₂ 
I = f(coords[0]) * weights[0] + f(coords[1]) * weights[1]

displayvar("I", I)
displayvar("error", (I - I_exact)/I_exact)


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Writing this as a loop instead

In [37]:

ngp = 5 # number of integration points
coords, weights = gauss_integration_rule(ngp)
# coords, weights = np.polynomial.legendre.leggauss(ngp)

I = 0 # set the integral value to zero 
for i in range(ngp):
    ξ = coords[i]
    W = weights[i]
    # displayvar(f"ξ_{i+1}", ξ)
    # displayvar(f"W_{i+1}", W)
    I += f(ξ) * W

displayvar("I", I)
displayvar("error", np.abs((I - I_exact)/I_exact), accuracy=3)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## 2D example

In [38]:
coords, weights = np.polynomial.legendre.leggauss(2)
displayvar("coords", coords)
displayvar("weights", weights)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Integration of a function in 2D over the parent domain

In [61]:
# Integrand 
def f_func(ξ, η):
    return 1 - (0.5*ξ + 0.2*ξ**2 + 0.4*η**3)

# Number of points per dimension
ngp = 1

coords, weights = gauss_integration_rule(ngp) 
# display(coords, weights)

# Compute integral
I_approx = 0.0
print(f"Gauss points and combined weights for ngp={ngp}:")
for i in range(ngp):
    for j in range(ngp):
        ξ = coords[i]
        η = coords[j]
        W = weights[i] * weights[j]  # product weight
        print(f"  point ({i+1},{j+1}): ξ={ξ:.4f}, η ={η:.4f}, W={W:.4f}")
        I_approx += f_func(ξ, η) * W

# Exact integral
I_exact = 56/15
error = (I_approx - I_exact) / I_exact

print("Approximate integral:", I_approx)
print("Exact integral      :", I_exact)
print("Relative error      :", error)


Gauss points and combined weights for ngp=1:
  point (1,1): ξ=0.0000, η =0.0000, W=4.0000
Approximate integral: 4.0
Exact integral      : 3.7333333333333334
Relative error      : 0.07142857142857141


## FE equations - bar element
Consider the element stiffness matrix for a 1D bar with shape functions in the parent domain given by
$N_1(\xi) = \frac{1-\xi}{2}$ and $N_2(\xi) = \frac{\xi+1}{2}$
$$
    K^e = \int_{x_1}^{x_2} \mathbf{B}^{eT} EA \mathbf{B}^e dx =
    \int_{-1}^{1} \mathbf{B}^{eT} EA \mathbf{B}^e \det(J) d\xi
$$

Compute this matrix using gauss integration

In [ ]:
E = 210e9
A = 1e-4

x1 = 2
x2 = 5
def compute_Be_detJ(ξ, x1, x2):
    dN1dξ = -0.5 
    dN2dξ = 0.5
    detJ = dN1dξ * x1 + dN2dξ * x2 # J = det(J) since J is a scalar
    dNdξ = np.array([[dN1dξ, dN2dξ]]) # derivatives wrt ξ 
    dNdx = 1/detJ * dNdξ # derivatives wrt ξ
    Be = dNdx
    return Be, detJ

ngp = 1 # number of integration points
coords, weights = np.polynomial.legendre.leggauss(ngp)
Ke = np.zeros((2, 2)) # set the integral value to zero 
for i in range(ngp):
    ξ = coords[i]
    W = weights[i]
    Be, detJ = compute_Be_detJ(ξ, x1, x2) 
    Ke += E*A * Be.T @ Be * W * detJ
displayvar("K^e", Ke)
displayvar("K^e_{spring}", spring1e(k=E*A/3))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Example from Lecture 9
Linear isoparametric triangle, computing Be

In [54]:
dNdξ = np.array([-1, 1, 0]) 
dNdη = np.array([-1, 0, 1]) 

nodes = np.array([
    [0, 0],
    [2, 0],
    [0, 1]
])


xe = nodes[:,0]
ye = nodes[:,1]

displayvar("xe", xe)
displayvar("ye", ye)


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [56]:

dxdξ = np.dot(dNdξ, xe)
dxdη = np.dot(dNdη, xe)
dydξ = np.dot(dNdξ, ye)
dydη = np.dot(dNdη, ye)

JT = np.array([
    [dxdξ, dxdη],
    [dydξ, dydη]
])
displayvar("J^T", JT)


<IPython.core.display.Math object>

In [58]:

JTinv = np.linalg.inv(JT)
dN = np.array([dNdξ, dNdξ]) 

displayvar("dNdξ", dN)
displayvar("dNdx", JTinv @ dN)
dNdx = JTinv@dN



<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [59]:

Be = np.array([
    [dNdx[0,0], 0, dNdx[0,1], 0, dNdx[0,2], 0],
    [0, dNdx[1,0], 0, dNdx[1,1], 0, dNdx[1,2]],
    [dNdx[1,0], dNdx[0,0], dNdx[1,1], dNdx[0,1], dNdx[1,2], dNdx[0,2]] 
])
displayvar("Be", Be)


<IPython.core.display.Math object>